In [1]:
from pynq.overlays.base import BaseOverlay
from pynq import Overlay
import time

In [2]:
# carrega o overlay base da fpga - permite interagir com leds, botões, etc
base = BaseOverlay("base.bit")
base_path = '/home/xilinx/'
current_ol = 'none'

In [2]:
# carrega o bitstream base principal do processador
def load_main_ol():
    ol = Overlay(base_path + 'black_parrot_zcu104_main.bit')

In [1]:
# devemos declarar qual região da FPGA foi marcada como reconfigurável com o 
# nome do bdf parcial usado naquele design, em nosso design temos 3 blocos

# bloco que reconfigura apenas a memória
def set_pr_mem():
    load_main_ol()
    Overlay.set_partial_region('block_mem')
    current_ol = "mem"

# bloco que reconfigura apenas a FPU
def set_pr_FPU():
    load_main_ol()
    Overlay.set_partial_region('block_FPU')
    current_ol = "FPU"

# bloco que reconfigura a memória e a FPU
def set_pr_both():
    load_main_ol()
    Overlay.set_partial_region('block_mem_and_FPU')
    current_ol = "both"

In [ ]:
#lemos a taxa de hit/miss do processador BlackParrot
trigger = ol.read_register('cache_rate')

In [ ]:
# podemos designar um botão para realizar a troca dos bitstream:
def load_ol_buttons():
    while True:
        if trigger >= 0.7:
            if current_ol != "mem":
                set_pr_mem()
            Overlay.download(base_path + 'black_parrot_zcu104_reduced_cache.bit')
            break
        
        time.sleep(0.5)
        trigger = ol.read_register('cache_rate')
        